# First Assignment for Data Science Course by Digital House

La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. 

<b> En este proyecto nos proponemos desarrollar un modelo de regresión que permita predecir el precio en dólares por metro cuadrado de una propiedad. El objetivo final, es desarrollar un tasador automático para ser aplicado a las próximas propiedades que sean comercializadas por la empresa.</b>

Para ello, la empresa le provee de un dataset correspondiente al primer semestre de 2017. 

Será importante tener en cuenta el problema de la <i>influencia espacial en los precios por metro cuadrado</i>. En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.


Objetivos de la primera etapa:

● Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.

● Realizar un análisis descriptivo de las principales variables.

● Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

Para la presentación en clase se deben armar algunos slides no técnicos para exponer en no más de 10 minutos (PPT o Google Slides). 
La presentación debe constar:
* una introducción (planteo del problema, la pregunta, la descripción del dataset, etc.) 
* un desarrollo de los análisis realizados (análisis descriptivo, análisis de correlaciones preliminares, visualizaciones preliminares) 
* una exposición de los principales resultados y conclusiones.

## Introducción

Problema: automatizar el proceso de predecir el precio por metro cuadrado de una propiedad en USD, creando así un tasador automático.

Entonces, ¿a qué features nos debemos basar para predecir el valor?


<img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DS0103EN/labs/images/lab1_fig2_datascience_methodology_flowchart.png" width = 500> 

Vamos a definir el enfoque analítico para resolver el problema, identificando qué tipo de patrones serán necesarios para abordar la cuestión de manera más efectiva. Aquí utilizaremos técnicas estadísticas y de aprendizaje automático.

## Preparación de datos

In [50]:
import pandas as pd
import numpy as np
df=pd.read_csv("properatti.csv")

In [51]:
df.shape

(121220, 26)

In [52]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [53]:
df.rename(columns={"Unnamed: 0":"a","operation":"operacion","property_type":"tipo_inmueble","place_name":"barrio","place_with_parent_names":"localizacion","country_name":"argentina","state_name":"localidad","geonames_id":"id_geoname","lat-lon":"coordenadas","lat":"latitud","lon":"longitud","price":"precio","currency":"moneda","price_aprox_local_currency":"precio_pesos","price_aprox_usd":"precio_usd","surface_total_in_m2":"sup_total","surface_covered_in_m2":"sup_cubierta","price_usd_per_m2":"precio_usd_sup","price_per_m2":"precio_pesos_sup","floor":"nivel","rooms":"ambientes","expenses":"expensas","properati_url":"url","description":"observaciones","title":"titulo","image_thumbnail":"url_imagen"}, inplace=True)

In [54]:
df.drop(columns="a", inplace=True)

In [55]:
df.head()

,operacion,tipo_inmueble,barrio,localizacion,argentina,localidad,id_geoname,coordenadas,latitud,longitud,...,sup_cubierta,precio_usd_sup,precio_pesos_sup,nivel,ambientes,expensas,url,observaciones,titulo,url_imagen
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [113]:
df.iloc[:1000, 10:27] ##filtrar los de usd vacios para ver lo de pesos 

,precio,moneda,precio_pesos,precio_usd,sup_total,sup_cubierta,precio_usd_sup,precio_pesos_sup,nivel,ambientes,expensas,url,observaciones,titulo,url_imagen
0,62000.0,USD,1093959.00,62000.00,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,150000.0,USD,2646675.00,150000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,72000.0,USD,1270404.00,72000.00,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,95000.0,USD,1676227.50,95000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,64000.0,USD,1129248.00,64000.00,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
5,NaN,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...
6,130000.0,USD,2293785.00,130000.00,106.0,78.0,1226.415094,1666.666667,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...
7,138000.0,USD,2434941.00,138000.00,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,195000.0,USD,3440677.50,195000.00,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
9,460000.0,ARS,455201.45,25798.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...


In [151]:
precios_vacios=(df.precio_pesos)/(df.precio_usd).isnull()
precios_vacios.to_frame()
precios_vacios.isnull().value_counts()

##tabla de frecuencias con histograma en la cual nos figure los datos nulos entre precios_usd y precios_pesos

##precios_usd y precios_usd_sup

False    100809
True      20411
dtype: int64

In [157]:
#valor de m2, precios en usd / sup , ver si coincide el real con el teórico

si hay que completar los datos que faltan 

si hay un campo 

In [56]:
df.describe()

,id_geoname,latitud,longitud,precio,precio_pesos,precio_usd,sup_total,sup_cubierta,precio_usd_sup,precio_pesos_sup,nivel,ambientes,expensas
count,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [57]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import datasets

In [58]:
nulos=df.isnull().sum()
pd.DataFrame(nulos)

,0
operacion,0
tipo_inmueble,0
barrio,23
localizacion,0
argentina,0
localidad,0
id_geoname,18717
coordenadas,51550
latitud,51550
longitud,51550


<h5> Completando valores nulos 

De dónde sacar los datos:

    -barrio: localidad, borrar
    -id_geoname: ¿qué utilidad tiene?
    -coordenadas, latitud y longitud: ver
    -precios usd es lo más completo, nos sirve para completar los 10000 faltantes dentro de superficie
    -de que sirve la imagen????? al día de hoy no podemos codificar imagenes y viceversa ----> eliminar
    -
    -
    -
    -

In [59]:
nulos.loc[["barrio","id_geoname","coordenadas","latitud","longitud","precio","moneda","precio_pesos","precio_usd","precio_pesos_sup","precio_usd_sup","sup_total","sup_cubierta","nivel","ambientes","expensas","url_imagen"]]

barrio                  23
id_geoname           18717
coordenadas          51550
latitud              51550
longitud             51550
precio               20410
moneda               20411
precio_pesos         20410
precio_usd           20410
precio_pesos_sup     33562
precio_usd_sup       52603
sup_total            39328
sup_cubierta         19907
nivel               113321
ambientes            73830
expensas            106958
url_imagen            3112
dtype: int64

In [60]:
df.observaciones[0]

'2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).'

In [61]:
df.observaciones[10]

'Hermosa casa sin terminar de 3 dormitorios, baño, living comedor, cocina, cochera.Techo de Madera. Con escritura y servicios. Lote de 1514 metros.                     Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).'

In [154]:
df.observaciones[12000]

'Venta de Departamento 3 AMBIENTES en Caballito.Bonifacio 1234 representa una excelente oportunidad de inversión en semipisos de 2 y 3 ambientes, sobre el radio más valorizado de Caballito. Un concepto de exclusividad con 14 unidades desarrolladas en 7 pisos, todas con disposición al frente,  recepcionando luz del norte con balcones aterrazados.El emprendimiento será construido en 24 meses, dando comienzo en mayo del corriente 2016 y teniendo la posibilidad de ingresar con un anticipo del 30% y financiar en dos años el saldo. Las unidades:Semipisos de 2 ambientes de 52.60 m2Semipisos de 3 ambientes de 71.03 m2Espacios comunes: Cocheras en planta baja optativas. No disponibles para la venta aún.Sector de laundry en octavo piso.Espacio común con terraza y parrilla. Solárium.Características técnicas:�Carpinterías exteriores de aluminio anodizado natural�Pisos de porcellanato en ambientes de living comedor, cocina y balcones. Baños en cerámicas esmaltadas. Piso flotante de 8 mm en dormitor

In [68]:
df.groupby("operacion").groups

{'sell': Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                  8,      9,
             ...
             121210, 121211, 121212, 121213, 121214, 121215, 121216, 121217,
             121218, 121219],
            dtype='int64', length=121220)}

In [70]:
df.groupby("barrio").groups

{' Country Maschwitz Club': Int64Index([33586, 42404, 45159], dtype='int64'),
 ' los alamos': Int64Index([23730], dtype='int64'),
 '9 de Abril': Int64Index([25553, 78527, 84843], dtype='int64'),
 'Abasto': Int64Index([   791,   5354,   5356,   5357,   5395,   5493,   7685,   8874,
               9055,  12804,
             ...
             110486, 110487, 110488, 110489, 110490, 110491, 110745, 116909,
             116992, 117867],
            dtype='int64', length=139),
 'Abril Club de Campo': Int64Index([  6186,   6188,   6189,  13098,  15086,  21942,  22998,  51748,
              57350,  65689,  66478,  68594,  70367,  72195,  72466,  72626,
              77640,  88181, 108625],
            dtype='int64'),
 'Acacias Blancas': Int64Index([ 13313,  14722,  19272,  29201,  33041,  46760,  47310,  54043,
              54045,  54381,  58919,  81726,  81882,  82161,  82325,  82918,
              83568,  85002,  85130,  85673,  86244,  87371,  87623,  90553,
              92408,  98102, 103

In [69]:
df.groupby("tipo_inmueble").groups

{'PH': Int64Index([     0,      3,      6,     16,     34,     46,     70,     71,
                 72,     80,
             ...
             120820, 120861, 120930, 120931, 120936, 120972, 121081, 121104,
             121130, 121148],
            dtype='int64', length=5751),
 'apartment': Int64Index([     1,      2,      4,      7,      8,     12,     13,     14,
                 17,     19,
             ...
             121203, 121204, 121205, 121206, 121209, 121211, 121215, 121217,
             121218, 121219],
            dtype='int64', length=71065),
 'house': Int64Index([     5,      9,     10,     11,     15,     18,     20,     22,
                 26,     27,
             ...
             121149, 121154, 121155, 121156, 121207, 121208, 121210, 121212,
             121213, 121216],
            dtype='int64', length=40268),
 'store': Int64Index([    57,     65,     95,    183,    204,    215,    275,    277,
                370,    492,
             ...
             120768, 1207

In [71]:
df.groupby("localizacion").groups ##

{'|Argentina|Bs.As. G.B.A. Zona Norte|': Int64Index([  1856,   2448,   3919,   4239,   5426,   8434,  10996,  10997,
              17176,  17558,
             ...
             114499, 117061, 117444, 117452, 117601, 117729, 118326, 118493,
             118819, 121141],
            dtype='int64', length=222),
 '|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|': Int64Index([    49,    955,   1188,   1189,   1190,   1345,   2557,   4050,
               4147,   4150,
             ...
             120942, 120943, 120947, 120948, 120950, 120952, 120953, 121101,
             121207, 121208],
            dtype='int64', length=1037),
 '|Argentina|Bs.As. G.B.A. Zona Norte|Escobar| Country Maschwitz Club|': Int64Index([33586, 42404, 45159], dtype='int64'),
 '|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|Acacias Blancas|': Int64Index([ 13313,  14722,  19272,  29201,  33041,  46760,  47310,  54043,
              54045,  54381,  58919,  81726,  81882,  82161,  82325,  82918,
              83568,  85002,  

In [74]:
df.pivot_table(index='localidad', aggfunc={'localidad':"count"})

,localidad
localidad,
Bs.As. G.B.A. Zona Norte,25560
Bs.As. G.B.A. Zona Oeste,9322
Bs.As. G.B.A. Zona Sur,13952
Buenos Aires Costa Atlántica,10006
Buenos Aires Interior,2291
Capital Federal,32316
Catamarca,27
Chaco,57
Chubut,259


In [92]:
df.pivot_table(index='tipo_inmueble', aggfunc={'tipo_inmueble':"count"})

,tipo_inmueble
tipo_inmueble,
PH,5751
apartment,71065
house,40268
store,4136


In [90]:
## ver los niveles más altos de 1000 y ambientes más de 30 QUE ONDA CON LOS IF
## VER REGEX QUE COINCIDA CON AMBIENTES Y TRAIGA EL NUMERO QUE ESTE AL LADO


In [96]:
df.pivot_table(index='ambientes', columns="tipo_inmueble", aggfunc={'ambientes':"count"})

ambientes                        
tipo_inmueble        PH apartment   house store
ambientes                                      
1.0               118.0    6940.0   264.0  55.0
2.0               744.0   10664.0  1008.0  17.0
3.0              1301.0    7946.0  3307.0  13.0
4.0               705.0    3867.0  3361.0   4.0
5.0               263.0    1162.0  2353.0   4.0
6.0                69.0     330.0  1002.0   2.0
7.0                24.0     174.0   566.0   6.0
8.0                10.0      86.0   312.0   1.0
9.0                 5.0      34.0   145.0   NaN
10.0                4.0      22.0   197.0   3.0
11.0                1.0       9.0    62.0   NaN
12.0                1.0      12.0    52.0   NaN
13.0                1.0       3.0    29.0   NaN
14.0                NaN       5.0    22.0   NaN
15.0                NaN       2.0    24.0   NaN
16.0                NaN       NaN    11.0   NaN
17.0                NaN       3.0    13.0   NaN
18.0                NaN       1.0     5.0   NaN
19.0                NaN       NaN     4.0   NaN
20.0                1.0       2.0     8.0   NaN
21.0                NaN       1.0     4.0   NaN
22.0                NaN       2.0     6.0   NaN
23.0                NaN       NaN     2.0   NaN
24.0                NaN       NaN     1.0   NaN
25.0                NaN       1.0     3.0   NaN
27.0                NaN       NaN     1.0   NaN
28.0                NaN       NaN     1.0   NaN
29.0                NaN       NaN     1.0   NaN
30.0                NaN       NaN     4.0   NaN
31.0                NaN       NaN     1.0   NaN
32.0                NaN       2.0     1.0   NaN

In [152]:

# precios = pd.cut(df['precio_pesos'], [0, 10000, 80000])
# df.pivot_table('tipo_inmueble', ['precio_usd', precios], 'precio_usd')


#los usd si la misma cant en registros en usd, hacer matriz 

In [165]:
#dedos.pivot_table(index='0', aggfunc={'0':"count"})

In [166]:
dedos=df.duplicated(['id_geoname']).to_frame()
dedos

,0
0,False
1,False
2,True
3,False
4,False
5,False
6,False
7,False
8,True
9,False


In [170]:
def reales(df.precio_usd_sup):
    for i in [df.precio_usd_sup]:
        if i==NaN 
        i=precio_usd/sup_total
        return i
    
    ## hacer lo mismo sup m2, precio m2, precio total usd
    ## 

SyntaxError: invalid syntax (<ipython-input-170-e21b8bb04ab6>, line 1)

In [169]:
reales(2)

holi


In [167]:
import re

In [ ]:
s=df.observaciones"ambientes"
re.findall(r'\bambientes\b', s)